<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-constants" data-toc-modified-id="Define-constants-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define constants</a></span><ul class="toc-item"><li><span><a href="#Define-paths" data-toc-modified-id="Define-paths-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Define paths</a></span></li></ul></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Load-features-and-fdump" data-toc-modified-id="Load-features-and-fdump-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load features and fdump</a></span></li><li><span><a href="#Load-NLP-data" data-toc-modified-id="Load-NLP-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load NLP data</a></span></li><li><span><a href="#Load-targets" data-toc-modified-id="Load-targets-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Load targets</a></span><ul class="toc-item"><li><span><a href="#Add-useful-columns" data-toc-modified-id="Add-useful-columns-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Add useful columns</a></span></li></ul></li></ul></li><li><span><a href="#Build-data" data-toc-modified-id="Build-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Build data</a></span><ul class="toc-item"><li><span><a href="#Drop-head" data-toc-modified-id="Drop-head-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Drop head</a></span></li><li><span><a href="#Join-NLP-features" data-toc-modified-id="Join-NLP-features-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Join NLP features</a></span></li><li><span><a href="#Join-targets-and-features" data-toc-modified-id="Join-targets-and-features-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Join targets and features</a></span></li><li><span><a href="#Fix-nans" data-toc-modified-id="Fix-nans-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Fix nans</a></span></li></ul></li><li><span><a href="#Save-reports-dataset" data-toc-modified-id="Save-reports-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Save reports dataset</a></span></li><li><span><a href="#Save-data" data-toc-modified-id="Save-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Save data</a></span></li></ul></div>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from arrow_utils import *

/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Define constants

In [3]:
VOLUME = 50
RES='5S'

## Define paths

In [4]:
target_save_folder = '../processed_data/targets/data_vol{}_bar_{}/'.format(VOLUME, RES)

In [5]:
os.listdir('../processed_data/targets/data_vol{}_bar_{}/'.format(VOLUME, RES))

['vb100_vf3_volatility_VWAP_200.pickle',
 'vb200_vf2_volatility_VWAP_200.pickle',
 'volatility_VWAP_200',
 'vb50_vf2_volatility_VWAP_200.pickle',
 'vb200_vf3_volatility_VWAP_200.pickle',
 'vb100_vf2_volatility_VWAP_200.pickle',
 'vb50_vf3_volatility_VWAP_200.pickle']

In [6]:
feature_save_folder = '../processed_data/features/data_vol{}_bar_{}/'.format(VOLUME, RES)
feature_saver = DataSaver(feature_save_folder)

target_save_folder = '../processed_data/targets/data_vol{}_bar_{}/'.format(VOLUME, RES)
target_folders=[target_save_folder + name +'/' for name in os.listdir(target_save_folder) if os.path.isdir(target_save_folder + name)]
target_savers = {k:DataSaver(k) for k in target_folders}

dataset_folder = '../training/data/data_vol{}_bar_{}/'.format(VOLUME, RES)
data_saver = DataSaver(dataset_folder)

# Load data

## Load features and fdump

In [7]:
feature_bar = feature_saver.load_volumebar()
fdump = feature_saver.load_fdump()

In [8]:
feature_bar.df.loc[(feature_bar.df.index<=datetime.datetime(2016,7,8)) & (feature_bar.df.index>=datetime.datetime(2012,7,8))].shape

(1457134, 229)

In [22]:
data = feature_bar.df[fdump['all_feats']]

## Load NLP data

In [23]:
nlp_feats = pd.read_pickle('./../processed_data/features/nlp_reports/nlp_features_with_bucket_ts.pkl')
nlp_feats.drop('Volume_bucket_ID', inplace=True, axis=1)

## Load targets

In [24]:
target_df_ls = [target_savers[k].load_df() for k in target_savers.keys()]
target_df = reduce(lambda left,right: left.join(right,how='outer'), target_df_ls)

### Add useful columns

In [25]:
idle_target_col = [c for c in target_df.columns if 'target_0' in c]
for c in idle_target_col:
    target_df[c.replace('target_0', 'target_not0')] = 1-target_df[c]
    target_df[c.replace('target_0', 'sample_weights_not0')] = (1-target_df[c])*target_df[c.replace('target_0', 'sample_weights')]

# Build data

## Drop head

In [26]:
rows_to_skip = fdump['rows_to_skip']

In [27]:
data = data.iloc[rows_to_skip+1:,]

## Join NLP features

In [28]:
nlp_feats.set_index('bucket_end', inplace=True)

In [59]:
fdump['feats']['nlp'] = list(nlp_feats.columns)
fdump['all_feats']+=list(nlp_feats.columns)

In [29]:
data = data.join(nlp_feats)

## Join targets and features

In [30]:
data = data.join(target_df)

## Fix nans

In [31]:
target_cols = [c for c in data.columns if ('target' in c) | ('sample_weights' in c) | ('futureReturn' in c)]

data[target_cols] = data[target_cols].fillna(0)

In [32]:
data['is_report'] = data['is_report'].fillna(False)

In [33]:
data.isnull().sum().to_frame('counts').query('counts>0')

,counts
Pre_Doc2Vec_f1,2458394
Pre_Doc2Vec_f2,2458394
Pre_Doc2Vec_f3,2458394
Pre_Doc2Vec_f4,2458394
Pre_Doc2Vec_f5,2458394
Pre_Doc2Vec_f6,2458394
Pre_Doc2Vec_f7,2458394
Pre_Doc2Vec_f8,2458394
Pre_Doc2Vec_f9,2458394
Pre_Doc2Vec_f10,2458394


In [34]:
data.tail()

,CrossBarVol_VWAP_100R,RelDelta_EWMA5R_VWAP,LogRet_1R_VWAP,MACD200R10R_Close,EWMA25R_NormCORange,elapsed_5_minutes,VolatilityDelta_EWMA50R_VWAP,SI_MPC_50,Sharpe_50R_Close,Sharpe_50R_FD_VWAP,...,target_not0_vb50_vf3_volatility_VWAP_200,sample_weights_not0_vb50_vf3_volatility_VWAP_200,target_not0_vb100_vf2_volatility_VWAP_200,sample_weights_not0_vb100_vf2_volatility_VWAP_200,target_not0_vb100_vf3_volatility_VWAP_200,sample_weights_not0_vb100_vf3_volatility_VWAP_200,target_not0_vb200_vf2_volatility_VWAP_200,sample_weights_not0_vb200_vf2_volatility_VWAP_200,target_not0_vb200_vf3_volatility_VWAP_200,sample_weights_not0_vb200_vf3_volatility_VWAP_200
bucket_end,,,,,,,,,,,,,,,,,,,,,
2017-10-13 21:46:00,0.046628,0.000403,0.000210,0.125327,0.000022,1305,0.759657,0.018104,0.816649,0.425613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-13 21:49:35,0.044337,0.000441,0.000103,0.127230,0.000021,1305,0.793079,0.000002,0.796660,0.349574,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-13 21:53:10,0.042685,0.000052,-0.000381,0.126582,-0.000002,1310,0.624544,-0.011077,0.227769,-0.133915,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-13 22:01:40,0.041387,0.000115,0.000080,0.126961,0.000006,1320,0.651657,0.017482,0.281880,-0.025162,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-13 22:29:05,0.040655,0.000576,0.000546,0.131908,0.000041,1345,0.871482,0.107314,0.782174,0.222183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Save reports dataset

In [38]:
report_data = data.query('is_report')

In [60]:
report_data.to_pickle('../training/data/reduced_data_vol50_bar_5S/data.pickle')

In [61]:
pd.to_pickle(fdump, '../training/data/reduced_data_vol50_bar_5S/fdump.pickle')

# Save data

In [63]:
fdump['target_cols'] = target_cols
data_saver.save(data=report_data, fdump=fdump)

Saving file 0....
Saving file 1....
Saving file 2....
Saving file 3....
Saving file 4....
Saving file 5....
Saving file 6....
Saving file 7....
Saving file 8....
Saving file 9....
Saving file 10....
Saving file 11....
Saving file 12....
Saving file 13....
Saving file 14....
Saving file 15....
Saving file 16....
Saving file 17....
Saving file 18....
Saving file 19....
Saving file 20....
Saving file 21....
Saving file 22....
Saving file 23....
Saving file 24....
Saving file 25....
Saving file 26....
Saving file 27....
Saving file 28....
Saving file 29....
Saving file 30....
Saving file 31....
SAVED!
